# Exoplanet Observation
 ### Yuelong Li, 07/17/2018
 ### Instructor: Gregory Tucker
  ####  1.  Simulate single planetary motion & Compare with Keplar's Law

* Import numpy library

In [ ]:
import numpy as np

* Construct celestrial body class
* state:

| $p$ | $v$   | $a$  |
|------|--------------|------|
| $x$ | $v_x$ | $a_x$ |
| $y$ | $v_y$ | $a_y$ |
| $z$ | $v_z$ | $a_z$ |
* Graviatation calculation:
    $F_{10} = (p_1-p_0)\frac{Gm_0m_1}{\|p_1-p_0\|^3}$
* Field variables needed:
    $G = 6.67*10^{-20}km kg^{-1} s^{-2}, m$


In [3]:
G = 6.67e-20
class CBody: #Definition of a celestrial body
    state = None # 3*3 matrix for state of motion (km)
    m = 1 # mass in kg
    
    def __init__(self, state, mass=0,theta=0,phi=np.pi/2, radius=0, speed=0):
        if(state == None):
            self.state=np.zeros((3,3))
            self.state[0] = np.array([
                np.sin(phi)*np.cos(theta)*radius, np.sin(phi)*np.sin(theta)*radius, np.cos(phi)*radius
            ])
            self.state[1] = np.array([
                -speed*np.sin(phi)*np.sin(theta), speed*np.sin(phi)*np.cos(theta), 0
            ])
        else:
            self.state = state.astype(float)
        self.m = mass
        return
        
    def step(self, dt): # propagate one step
        for i in range(2): # Increment all states of motion by once
            self.state[i]+=dt*self.state[i+1]
        self.state[2] = 0 # Clear acceleration vector
        return self.state
    
    def pullEachOther(self,cbody): # compute mutual attraction
        p01 = (self.state[0]-cbody.state[0]) #Vector pointing from another cbody to this one
        F = p01*(G*self.m*cbody.m/(np.linalg.norm(p01)**3)) # Newton's Law of gravitation
        cbody.state[2]+=F/cbody.m
        self.state[2]-=F/self.m

* Test Cbody stepping method
* Earth mass = 5.972e24kg DtoSun=149,597,870 km Speed = 29.783 km/s
* Jupiter mass = 1.898e27kg DtoSun=778,547,200 km Speed = 13.06 km/s
* Venus mass = 4.867e24kg DtoSun=108,200,000 km Speed = 35.03 km/s
* Sun mass = 1.989e30kg DtoSun=0 km Speed = 0 km/s

In [4]:
earth = CBody(mass=5.972e24, radius = 14957870, theta = 3, speed = 29.783)
print(earth.state)
jupiter = CBody(mass = 1.898e27, radius=778547200, theta = 40, speed = 13.06)
print(earth.state)
venus = CBody(mass = 4.867e24, radius=108200000, theta = 25, speed = 35.03)
sun = CBody(mass = 1.989e30, radius = 0, speed = 0, theta = 180)
bodies = [earth, jupiter,sun]

def stepAll(bodies, dt):
    stateCapture = np.empty([len(bodies),3,3])
    i = 0
    for body in bodies:
        body.step(dt)
        stateCapture[i] = (body.state)
        i+=1
    return stateCapture

stepAll(bodies,10)

TypeError: __init__() missing 2 required positional arguments: 'state' and 'phi'

* Test Force Interaction Method

In [75]:
def computeRelations(bodies):
    i = 1
    for cbody in bodies:
        for j in range(i,len(bodies)):
            cbody.pullEachOther(bodies[j])
        i+=1

computeRelations(bodies)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in double_scalars
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in multiply


  #### 2. Visualize Objects

In [23]:
totalSteps = 10000
stateRecord = np.empty([totalSteps,len(bodies),3,3])
for i in range(totalSteps):
    computeRelations(bodies)
    stateRecord[i] = stepAll(bodies,0.01)


* Experiment with matplot lib

* Experiment with dot product and transpose

In [ ]:
 a = np.arange(120).reshape((2,3,4,5,))
np.transpose(a,(1,2,3,0))

* Transpose the actual computed data, change shape from [100,len(bodies),3,3] to [len(bodies),3, 3, 100]
so that they can be plotted by matplot

In [25]:
visualization = np.transpose(stateRecord,(1, 2, 3, 0))
print(visualization)

[[[[-2.00000000e+00 -1.99982756e+00 -1.99956890e+00 ... -1.73185420e+01
    -1.73165298e+01 -1.73145149e+01]
   [ 3.00000000e-02  4.49993364e-02  5.99980093e-02 ...  3.23071058e+01
     3.23130487e+01  3.23189913e+01]
   [-2.00000000e-03 -2.99994002e-03 -3.99982006e-03 ...  1.82917711e+00
     1.82906349e+00  1.82894992e+00]]

  [[ 1.72443707e-02  2.58657653e-02  3.44863856e-02 ...  2.01215915e-01
     2.01496907e-01  2.01777912e-01]
   [ 1.49993364e+00  1.49986729e+00  1.49976779e+00 ...  5.94292927e-01
     5.94262373e-01  5.94231788e-01]
   [-9.99940017e-02 -9.99880041e-02 -9.99790088e-02 ... -1.13623131e-02
    -1.13568929e-02 -1.13514690e-02]]

  [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
     0.00000000e+00  0.00000000e+00]
   [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
     0.00000000e+00  0.00000000e+00]
   [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
     0.00000000e+00  0.00000000e+00]]]


 [[[ 5.0000

In [10]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

mpl.rcParams['legend.fontsize'] = 10

fig = plt.figure()
ax = fig.gca(projection='3d')

ax.plot(visualization[0][0][0],visualization[0][0][1],visualization[0][0][2], label='earth')
ax.plot(visualization[1][0][0],visualization[1][0][1],visualization[1][0][2], label='jupiter')
ax.plot(visualization[2][0][0],visualization[2][0][1],visualization[2][0][2], label='sun')
ax.legend()

plt.show()
plt.interactive(True)

NameError: name 'visualization' is not defined